In [ ]:
import pandas as pd

In [ ]:
# Read parquet files
data_camera: pd.DataFrame = pd.read_parquet('s3://datalake-eu-central-1/ugiO-atchackathon/raw/other/camera_2022_09_01.parquet')
data_radar: pd.DataFrame = pd.read_parquet('s3://datalake-eu-central-1/ugiO-atchackathon/raw/radar/radar_count_2022_09_01.parquet')

# Column preprocessing
data_camera["time_index"] = pd.to_datetime(data_camera["time_index"])
data_camera = data_camera.set_index("time_index")
data_radar["time_index"] = pd.to_datetime(data_radar["time_index"])
data_radar = data_radar.set_index("time_index")

# Add pX col
data_radar["n_p"] = data_radar["entity_id"].str.split(pat=":", expand=True)[4]
data_camera["n_p"] = data_camera["entity_id"].str.split(pat=":", expand=True)[4]

In [ ]:
data_camera_proc = []
data_radar_proc = []

# Preprocess camera data
for idx, df_loop in data_camera.groupby(["n_p"]):
    # Only filter by person records
    data_aux = df_loop[df_loop["class_label"] == "person"]

    # Resample
    data_aux = data_aux.resample('1T').mean()
    data_aux = data_aux.resample('15T').sum()
    
    data_aux["n_p"] = idx

    data_camera_proc.append(data_aux)

data_camera_proc = pd.concat(data_camera_proc)


# Preprocess radar data
for idx, df_loop in data_radar.groupby(["n_p"]):
    # Resample (some we need the mean and another we need the sum)
    data_aux = df_loop.resample('15T').mean()
    data_aux2 = df_loop.resample('15T').sum()

    cols_sum = ["vehiclelight", "vehicleheavy", "vehicleothers"]
    data_aux[cols_sum] = data_aux2[cols_sum]
    
    data_aux["n_p"] = idx

    data_radar_proc.append(data_aux)

data_radar_proc = pd.concat(data_radar_proc)

In [ ]:
df_agg = data_radar_proc.merge(data_camera_proc, on=["time_index", 'n_p'], how="inner")

In [ ]:
df_agg

In [ ]:
df_agg.to_parquet('s3://datalake-eu-central-1/ugiO-atchackathon/preprocessed/aggregated.parquet')